## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import fft
import plotly.graph_objects as go
from scipy import signal
import os

## MAFAULDAS values

In [ ]:
fs = 50000 #[Hz] frequencia de amostragem
T_amost =5 #[s]  tempo total de amostragem
Ts=1/fs    #[s]  Periodo de amostragem  
t = np.arange(0,T_amost,Ts) # vetor tempo de 0 a 5s

## Functions

In [ ]:
# function to read and set columns of MAFAULDA dataset
def read_file(file):
  columns = ['tacometro', 'axial_1','radial_1', 'tangencial_1','axial_2','radial_2', 'tangencial_2','microfone']
  csv = pd.read_csv(file, header=None, names=columns)
  sos = signal.butter(N=10, Wn=[10,5500], btype='bandpass', fs=fs, output='sos')
  for col in csv.columns[1:-1]:
    csv[col] = signal.sosfilt(sos, csv[col])
  return csv

#filter tacometer signal
def filtro(sinal, low=3, high=3):
  sinal[sinal<low]=0
  sinal[sinal>high]=5
  return sinal

#return the rotation(Hz) from the tacometer signal
def get_rotation(sinal, n=10):
  picos, count, temp, start = 0, 0, 5, False
  sinal= filtro(sinal) 
  for value in sinal:
    if (temp==0) and (value== 5.0):
      start=True
      picos+=1
      if picos > n: break
    temp = value
    if start: count+=1
  return  n/(count*Ts)

def find_rotation(sinal, h=0.5):
  return signal.find_peaks(to_fft(sinal.values),height=h)[0][0]/ (sinal.shape[0]/fs)   

# return the fft 
def to_fft(y):
  return np.abs(fft.fft(y))[0:y.shape[0]//2] / (y.shape[0]//2)

#plots a FFT for MAFAULDAS data
def plot_fft(df:pd.DataFrame, plot_cols:list, title=None):
  fig = go.Figure()
  for col in plot_cols:
    y_fft= to_fft(df[col].values)
    fig.add_trace(go.Scatter(
        x=fft.fftfreq(df.shape[0],Ts),
        y= y_fft,
        name = col
    ))
  if title !=None:
    fig.update_layout(title=title)
  fig.show()

#--------------Stats from time domain-------------------
def peak(x):
  return np.max(abs(x))
def rms(x):
  return np.sqrt(np.mean(x**2))
def crista(x):
  return peak(x)/rms(x)
def curtose(x):
  a = 1/x.shape[0] * np.sum((x-np.mean(x))**4)
  b = (1/x.shape[0] * np.sum((x-np.mean(x))**2))**2
  return a/b
#------------------------------------------------------

#Peaks in FFT
def get_peaks(fft, freqs, rot, delta=3):
  values = []
  xvar = fft.shape[0]//(fs//2)
  ref = (np.array(freqs)*rot*xvar).astype(int)
  for freq in ref:
    values.append(np.max(fft[freq-delta:freq+delta+1]))
  return values

#Split df in n parts-----------------------------------
def split_df(df, n):
  df_splited = []
  y= df.shape[0]//n
  for x in range(n):
    df_splited.append(df.iloc[x*y:(x+1)*y])
  return df_splited


def split_df_2(df, n=5):
  df_splited = []
  for sample in split_df(df, n):
    df_splited.append(sample.iloc[:,1:4])
    df_splited.append(sample.iloc[:,4:7])

  return df_splited


# Load, Get Rotation, and Plot FFT

## Carregar do drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#for files in drive
drive_files = ['imbalance/15g/59.392.csv'] #insert list of files names .csv
plot_cols = ['axial_1','radial_1','tangencial_1'] # list of cols to plot

for file in drive_files:
  df = read_file(PATH+file)
  rot = get_rotation(df.tacometro)
  title = f'{file[:-4]} ({rot:.2f} Hz)'
  plot_fft(df,plot_cols, title)

In [ ]:
i= 2 # 0-axial, 1-radial, 2-tangencial
fig = go.Figure()
for file in drive_files:
  df= read_file(PATH+file)
  y_fft=to_fft(df[plot_cols[i]].values)
  fig.add_trace(go.Scatter(
                x= fft.fftfreq(df.shape[0],Ts),
                y=y_fft,
                name=file.split('/')[0] +' '+ plot_cols[i]
                )) 
fig.show()

## Carregar do PC

In [ ]:
from google.colab import files
fnames = list(files.upload())

Saving 40.5504.csv to 40.5504.csv


In [ ]:
#for files from PC
plot_cols = ['axial_2','radial_2','tangencial_2'] # list of cols to plot

for file in fnames:
  df = read_file(file)
  rot = find_rotation(df.tacometro)
  title = f'{file[:-4]} ({rot:.2f} Hz)'
  plot_fft(df,plot_cols, title)